In [175]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
from keras.models import load_model
import pandas as pd
import numpy as np
from PIL import Image
from keras.applications.mobilenetv2 import MobileNetV2, preprocess_input
from sklearn.metrics import accuracy_score, roc_auc_score, precision_recall_curve, auc, f1_score
from tqdm import tqdm
import scikitplot as skplt
from sklearn.model_selection import train_test_split
import glob
from sklearn.utils.class_weight import compute_class_weight
import shutil

In [123]:
stylegan = pd.read_csv("stylegan.csv")

In [4]:
model = load_model("../../models/dataset-4-skincancer/comparaison/mobilnet_224x224_weighted/best_model_epoch.hdf5")

In [6]:
model_cat = load_model("../../models/dataset-4-skincancer/comparaison-cat/Mobilnet_ACGAN_weighted/best_model_epoch.hdf5")

In [124]:
new_seq_3 = []
for row in tqdm(stylegan.iterrows()):
    
    n = row[1]["name"]
    
    im = Image.open("/home/yannis/Developpement/ppd-GAN-for-medical-imaging/dataset-4-skin-cancer/result_gan/stylegan/"+n)
    img = im.resize((224,224))
    img = preprocess_input(np.array(img).astype(np.float32))
    img = img.reshape(1,224,224,3)
    
    res = dict(row[1])
    p1 = model.predict(img)
    
#     if np.argmax(p1, axis=1)[0] == 1:
    res["label"] = p1[0]

    p1 = model_cat.predict(img)
    res["label_cat"] = p1[0]

    new_seq_3.append(res)

50000it [16:11, 53.99it/s]


In [125]:
all_data = []
all_data.extend(new_seq_3)
all_data.extend(new_seq_2)

df = pd.DataFrame(all_data)

In [126]:
lb = np.array([x for x in df["label"]])
label = np.argmax(lb, axis=1)
value_label = np.amax(lb, axis=1)
df["label"] = label
df["label_value"] = value_label

In [127]:
lb = np.array([x for x in df["label_cat"]])
label = np.argmax(lb, axis=1)
value_label = np.amax(lb, axis=1)
df["label_cat"] = label
df["label_cat_value"] = value_label

In [128]:
df.head()

,label,label_cat,name,psi,randomize_noise,seed,snapshot,label_value,label_cat_value
0,0,2,seed_6146640-lb_1--c85fde0e.png,NaN,NaN,6146640,9726,0.927319,0.743303
1,0,5,seed_2556030-lb_1--6d23bf87.png,NaN,NaN,2556030,9726,0.999997,1.000000
2,0,5,seed_3389776-lb_1--623752fe.png,NaN,NaN,3389776,9726,1.000000,1.000000
3,0,5,seed_970838-lb_1--a6d07fa3.png,NaN,NaN,970838,9726,0.668097,0.999944
4,0,5,seed_6888564-lb_1--f5dd9599.png,NaN,NaN,6888564,9726,0.639979,0.993624


In [129]:
df["label"].value_counts()

0    44935
1     9685
Name: label, dtype: int64

In [130]:
df["label_cat"].value_counts()

5    43747
4     6337
1     2227
2     1118
0      703
6      478
3       10
Name: label_cat, dtype: int64

In [131]:
df.to_csv("stylegan_labelised.csv", index=False)

In [186]:
df = pd.read_csv("stylegan_labelised.csv")

In [187]:
d0 = df[df["label"] == 0]
d0 = d0[d0["label_value"] > 0.8]

d1 = df[df["label"] == 1]
d1 = d1[d1["label_value"] > 0.65]

train_test = d0.append(d1)

# train, test = train_test_split(train_test, stratify = train_test["label"],test_size=0.10)

In [188]:
train_test.to_csv("st_labelised__train.csv", index=False)
# test.to_csv("st_labelised__test.csv", index=False)

In [189]:
compute_class_weight('balanced',np.unique(train_test["label"]),train_test["label"])

array([0.59633188, 3.09519487])

In [191]:
df = pd.read_csv("stylegan_labelised.csv")
d0 = df[df["label_cat"] == 5]
d0 = d0[d0["label_cat_value"] > 0.8]

d1 = df[df["label_cat"] != 5]
d1 = d1[d1["label_cat_value"] > 0.65]

train_test_cat = d0.append(d1)
train_test_cat["label_cat"].value_counts()

# train_cat, test_cat = train_test_split(train_test_cat, stratify = train_test_cat["label_cat"],test_size=0.10)

train_test_cat.to_csv("st_labelised_cat__train.csv", index=False)
# train_cat.to_csv("st_labelised_cat__test.csv", index=False)

In [192]:
compute_class_weight('balanced',np.unique(train_test_cat["label_cat"]),train_test_cat["label_cat"])

array([1.41132337e+01, 3.82882680e+00, 8.88101045e+00, 1.82060714e+03,
       1.29857856e+00, 1.74659517e-01, 1.68574735e+01])